## Train mallet lda model using best tuning results, convert to gensim model, and save LDAvis and excel file

#### Some terminologies:
1. raw_doc: unprocessed raw document from txt file
2. docs: lemmentized corpus
3. corpus_bow: bag of words corpus
4. corpus_tfidf: tfidf corpus

#### Change from eariler version:
1. filter out documents with too few words
2. use mallet

### Load dictionary and pre-built functions

In [12]:
from gensim import corpora, models
import numpy as np
import sys
import os
import gensim
import pickle
#from collections import Counter
#import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import pyLDAvis
import pyLDAvis.gensim as gensim_vis

In [2]:
## global folder path 
data_folder = '../../data/'
model_folder = '../../model/'
raw_data_path = os.path.join(data_folder,'raw/article_IV_corpus.txt')
data_processed_folder = os.path.join(data_folder,'processed')
results_folder = os.path.join(data_folder,'results','temp_results')
## binary file for mallet model
mallet_path = '/mnt/packages/Mallet/bin/mallet' # update this path

In [3]:
# %load topic_models.py
# python_root = './scripts'
# sys.path.insert(0, python_root)

#%%
def prepare_data(data_folder,save=True):
    ## read and transform data 
    contents = pickle.load(open(os.path.join(data_folder,'lemma_corpus.p'), "rb"))
    print('length of lemmentized corpus: {}'.format(len(contents)))
    docs = list()
    for paragraph in contents:
        docs.append([w for sentance in paragraph for w in sentance])

    # build dictionary

    dictionary = corpora.Dictionary(docs)
    dictionary.filter_extremes(no_below=5,no_above=0.5, keep_n=10000)
    # convert document into bow
    corpus_bow = [dictionary.doc2bow(text) for text in docs]
    ## comput tfidf feature vectors
    tfidf = models.TfidfModel(corpus_bow) # smartirs = 'atc' https://radimrehurek.com/gensim/models/tfidfmodel.html
    corpus_tfidf = tfidf[corpus_bow]
    
    ## save dictionary and corpora 
    if save:
        dictionary_save_path = os.path.join(data_folder,'dictionary.dict')
        dictionary.compactify()
        dictionary.save(dictionary_save_path)
        corpora.MmCorpus.serialize( os.path.join(data_folder,'corpus_bow.mm'), corpus_bow)
        corpora.MmCorpus.serialize( os.path.join(data_folder,'corpus_tfidf.mm'), corpus_tfidf)
        #print(len(dictionary))
    return docs,dictionary,corpus_bow,corpus_tfidf

#%%

### Load and process data

#### Load original text to look through later

In [4]:
with open(raw_data_path,'r',encoding='utf8') as f:
    raw_doc = f.readlines()
    raw_doc = [l.strip(' \n') for l in raw_doc if len(l)>50]

print('Length of raw documents {}'.format(len(raw_doc)))

Length of raw documents 142564


#### Load lemmentized corpus

In [5]:
rerun = True
if rerun == True:                ## save gensim objects, corpus, dictionary, and lda model
    mode = 'all'
    docs,dictionary,corpus_bow,corpus_tfidf = prepare_data(data_processed_folder,save=False)
    # corpus_bow = [c for c in corpus_bow_full if len(c)>0]
    
print('Length of length of bag-of-word corpus: {}'.format(len(corpus_bow)))

length of lemmentized corpus: 142564
Length of length of bag-of-word corpus: 142564


#### filter out paragraphs with <20 words or contain 'titles'

In [6]:
raw_doc_new = list()
corpus_bow_new = list()
docs_new = list()

tuple_temp = [(x, y, z) for (x, y, z) in zip(raw_doc, corpus_bow, docs) if len(x.split())>20 and ('<Title>' not in x) ]

raw_doc_new, corpus_bow_new, docs_new = zip(*tuple_temp)

print('Length of corpus without "<Title>" and has more than 20 words: {}'.format(len(raw_doc_new)))

raw_doc_new[0]

Length of corpus without "<Title>" and has more than 20 words: 123908


'1. The Russian economy proved to be more resilient than expected to the dual shocks of lower oil prices and sanctions. Output fell sharply in 2015, by 2.8 percent (revised from an initial estimate of 3.7 percent) but stabilized in 2016, contracting by only 0.2 percent. The relatively modest response to the large external shocks reflects the authorities’ effective policy response—floating exchange rate, banking system liquidity support and capital injections, and limited fiscal stimulus coupled with restrictive incomes policies—and was enabled by robust buffers.'

### Train Mallet LDA model

In [7]:
n_topics = 50
n_words = 20
np.random.seed(seed=1)
lda_mallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_bow_new, num_topics=n_topics, 
                                             id2word= dictionary, alpha= 1, optimize_interval=10, 
                                             iterations = 2000, random_seed = 1, 
                                             prefix=os.path.join(model_folder,"mallet_{}_topics_".format(n_topics)))
#initial alpha = 5/ n_topics = 5/ 50 = 0.1

In [8]:
lda_gensim = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_mallet)

In [ ]:
print('calculating coherence socre for {} documents ......'.format(len(docs_new)))
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=docs_new, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### Visualization

In [13]:
# Test if gensim model conversion gives the same viz- slight difference in ranking due to topic probability. However, positions are correct.
pyLDAvis.enable_notebook()
vis_data = gensim_vis.prepare(lda_gensim,corpus= corpus_bow_new, dictionary= dictionary)
vis_data

/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/usr/local/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
20    -0.118356  0.014992       1        1  3.852021
36    -0.042449  0.047656       2        1  3.805021
13    -0.186275 -0.154658       3        1  3.679474
1      0.249219  0.036087       4        1  3.565814
32     0.174007 -0.053990       5        1  3.532097
14     0.118477 -0.185839       6        1  3.306867
22    -0.117740 -0.045479       7        1  3.223949
40    -0.079396  0.200753       8        1  2.971378
38     0.243369  0.062616       9        1  2.600069
25    -0.091922  0.049911      10        1  2.580985
3      0.006316  0.184085      11        1  2.397134
35    -0.031695 -0.020788      12        1  2.384238
45     0.228889  0.008939      13        1  2.363024
27     0.097368 -0.009450      14        1  2.339445
21     0.041954 -0.048787      15        1  2.326364
2      0.125619  0.210271      16        1  2.307941
29    -0.257460  0.054330      17        1  2.106629
33    -0.241316 -0.087013      18        1  2.049720
23    -0.163335  0.072707      19        1  2.018670
24     0.015326  0.191123      20        1  1.973629
16    -0.254676  0.072890      21        1  1.964436
4     -0.148559 -0.155188      22        1  1.912416
44     0.159875  0.187460      23        1  1.874416
30    -0.048163  0.060874      24        1  1.830570
49     0.041474 -0.236562      25        1  1.829706
9      0.156809 -0.082698      26        1  1.813459
7     -0.153183 -0.071089      27        1  1.709325
5     -0.222276  0.090858      28        1  1.682147
0      0.253562  0.020530      29        1  1.627915
8      0.165477 -0.107764      30        1  1.599928
11     0.146232 -0.016239      31        1  1.570264
28    -0.073973  0.129014      32        1  1.556055
48    -0.076891  0.019624      33        1  1.544482
47     0.051253 -0.002946      34        1  1.504423
31     0.242188 -0.007531      35        1  1.490850
19    -0.073299 -0.196328      36        1  1.472646
42     0.159603 -0.113252      37        1  1.460225
15    -0.103406 -0.136114      38        1  1.426339
46     0.098277  0.075808      39        1  1.423507
34     0.024714  0.041093      40        1  1.409615
18    -0.152500  0.021450      41        1  1.392120
43     0.020695  0.013096      42        1  1.338221
37     0.196285 -0.134909      43        1  1.334923
41    -0.076872 -0.048857      44        1  1.301814
6     -0.087159 -0.025661      45        1  1.297943
17    -0.113875  0.070331      46        1  1.219229
26    -0.073893  0.022622      47        1  1.155092
10     0.174758  0.073797      48        1  1.124216
39    -0.139271  0.051574      49        1  0.900821
12    -0.063804 -0.143345      50        1  0.848424, topic_info=     Category          Freq             Term         Total  loglift  logprob
909   Default  43091.000000             bank  43091.000000  30.0000  30.0000
162   Default  43919.000000              gdp  43919.000000  29.0000  29.0000
20    Default  44334.000000           fiscal  44334.000000  28.0000  28.0000
426   Default  23522.000000              tax  23522.000000  27.0000  27.0000
6     Default  69812.000000        authority  69812.000000  26.0000  26.0000
418   Default  24262.000000          revenue  24262.000000  25.0000  25.0000
166   Default  19683.000000        inflation  19683.000000  24.0000  24.0000
163   Default  38717.000000       government  38717.000000  23.0000  23.0000
188   Default  22143.000000          deficit  22143.000000  22.0000  22.0000
16    Default  15372.000000    exchange_rate  15372.000000  21.0000  21.0000
117   Default  36328.000000           growth  36328.000000  20.0000  20.0000
94    Default  15000.000000             wage  15000.000000  19.0000  19.0000
471   Default  16548.000000            price  16548.000000  18.0000  18.0000
374   Default  31053.000000           reform  31053.000000  17.0000  17.0000
386   Default  20821.000000           budget  

In [14]:
id_map = dict(zip(np.array(vis_data.topic_order)- 1  , list(range(1,51))))
# key: topic_id_gensim + 1 , value: topic_id_LDAvis (ranked by topic size)

### Save Model and Results

In [124]:
def save_results_to_excel(today, model = lda_gensim, corpus_bow_new = corpus_bow_new, n_topics = n_topics, n_words = n_words, id_map = id_map):
    
    '''saves results to excel for better visualization'''
    # 1. Make a topic-key word table
    
    topic_word_df = pd.DataFrame(data = np.zeros((n_topics, n_words)), columns= ['word'+ str(x) for x in range(n_words)])
    for i in range(n_topics):
        topic_word_df.iloc[i] = pd.DataFrame(model.show_topic(topicid= i, topn= n_words))[0].tolist()
    topic_word_df['viz_topic_id'] = [id_map[key] for key in topic_df.index.to_list()]
    
    # 2. Make dataframe for doc_topic using parallel computing
    
    import time
    start_time = time.time()
    doc_topic = [*map(lambda x: model.get_document_topics(x), corpus_bow_new)]
    print("--- %s seconds ---" % (time.time() - start_time))

    doc_topic_df = pd.DataFrame(doc_topic)
    doc_topic_df['paragraph_id'] = doc_topic_df .index.to_list()
    doc_topic_df  = doc_topic_df.melt(id_vars='paragraph_id')
    doc_topic_df = doc_topic_df[doc_topic_df['value'].notnull()]
    doc_topic_df['gensim_topic'] = doc_topic_df['value'].apply(lambda x: x[0])
    doc_topic_df['probability'] = doc_topic_df['value'].apply(lambda x: x[1])
    doc_topic_df  = doc_topic_df[['paragraph_id','gensim_topic', 'probability']]
    doc_topic_df['viz_topic_id']= doc_topic_df ['gensim_topic'].apply(lambda x: id_map[x])

    # 3. Make paragraph table
    
    paragraph_df = pd.DataFrame(columns=['Paragraph'], data = raw_doc_new)
    
    # 4. save results to excel
    
    writer = pd.ExcelWriter(path = os.path.join(results_folder,'Mallet_{}_topics_{}.xlsx'.format(n_topics, today)))
        
    topic_word_df.to_excel(writer, 'Toipc and Key Word')
    doc_topic_df.to_excel(writer, 'Document and Topic')
    paragraph_df.to_excel(writer,'Paragraph')
    
    writer.save()  
    
    return

In [ ]:
now = datetime.datetime.now()
now = now.strftime("%Y_%m_%d")

lda_gensim.save(os.path.join(model_folder,'mallet_as_gensim_weights_{}_{}'.format(n_topics, now)))
# test loading model
#from gensim.models import LdaModel
#test = LdaModel.load(os.path.join(model_folder,'mallet_as_gensim_weights_{}_{}'.format(n_topics, now)))

pyLDAvis.save_html(vis_data,  os.path.join(results_folder,'mallet_as_gensim_{}_topics_{}.html'.format(n_topics, now))) 

save_results_to_excel(today = now)

--- 73.21768116950989 seconds ---
